In [1]:
import ray
from ray import tune
import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo.appo import DEFAULT_CONFIG
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv
from pybullet_envs.gym_locomotion_envs import Walker2DBulletEnv
from ray.tune.registry import register_env
import pybullet_envs



def five_link_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance


def bullet_walker_creator(env_config):
    return Walker2DBulletEnv()  # return an env instance

register_env("five_link-v3", five_link_creator)
register_env("Walker2DBulletEnv-v0", bullet_walker_creator)





/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/s

In [ ]:
config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 1
config["num_workers"] = 15
config["eager"] = False
config["model"]["fcnet_hiddens"] = [64,64]
config["lr"] = 5e-5
config["env"] = "Walker2d-v3"
#config["env"] = "Walker2DBulletEnv-v0"
config['observation_filter'] = 'MeanStdFilter'
ray.init()
analysis = tune.run(
    "PPO",
    config = config,
    stop={"timesteps_total": 1e7},
)

2019-11-06 15:21:56,171	INFO resource_spec.py:205 -- Starting Ray with 4.2 GiB memory available for workers and up to 2.12 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/4.2 GiB heap, 0.0/1.42 GiB objects
Memory usage on this node: 7.5/15.7 GiB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 16/16 CPUs, 1/1 GPUs, 0.0/4.2 GiB heap, 0.0/1.42 GiB objects
Memory usage on this node: 7.6/15.7 GiB
Result logdir: /home/sgillen/ray_results/PPO
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - PPO_Walker2d-v3_0:	RUNNING

(pid=7688) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=7688)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=7688) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in

In [1]:
import ray
from ray import tune
import ray.rllib.agents.ars as ars
import pybullet_envs


config = ars.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 14
config["eager"] = False
config["model"]["fcnet_hiddens"] = []
config["lr"] = 5e-5
config["env"] = "Walker2d-v3"
#config["env"] = "Walker2DBulletEnv-v0"

config['seed'] = tune.grid_search([1])

ray.init()
analysis = tune.run(
    "ARS",
    config = config,
    stop={"timesteps_total": 2e6},
)

/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/s

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/5.13 GiB heap, 0.0/1.76 GiB objects
Memory usage on this node: 6.3/15.7 GiB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 15/16 CPUs, 0/1 GPUs, 0.0/5.13 GiB heap, 0.0/1.76 GiB objects
Memory usage on this node: 6.4/15.7 GiB
Result logdir: /home/sgillen/ray_results/ARS
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - ARS_Walker2d-v3_0_seed=1:	RUNNING

(pid=3798) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3798)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=3798) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depre

2019-11-06 13:50:06,252	INFO tune.py:274 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.
2019-11-06 13:50:06,255	WARNING experiment_analysis.py:34 -- pandas not installed. Run `pip install pandas` for Analysis utilities.


Result for ARS_Walker2d-v3_0_seed=1:
  date: 2019-11-06_13-50-06
  done: true
  episode_len_mean: 1000.0
  episode_reward_mean: 995.4634368896484
  experiment_id: 77c6d434963a43d5b139012b9cde7644
  hostname: sgdt
  info:
    episodes_so_far: 3864
    episodes_this_iter: 56
    grad_norm: 9.00768756866455
    update_ratio: 0.07657606154680252
    weights_norm: 0.155051127076149
    weights_std: 0.03750545158982277
  iterations_since_restore: 69
  node_ip: 192.168.0.12
  perf:
    cpu_util_percent: 81.92
    gpu_util_percent0: 0.008
    ram_util_percent: 59.9
    vram_util_percent0: 0.12757201646090535
  pid: 3798
  time_since_restore: 151.83171033859253
  time_this_iter_s: 3.927502393722534
  time_total_s: 151.83171033859253
  timestamp: 1573077006
  timesteps_since_restore: 2023176
  timesteps_this_iter: 55361
  timesteps_total: 2023176
  training_iteration: 69
  trial_id: 04919da6
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/5.13 GiB h